# Projeto - Natural Language Processing (NLP)

**Integrantes**:
- Antonio Lucas Michelon de Almeida
- Pedro Nery Affonso dos Santos
- Rafael Gordon Paves

In [ ]:
import pandas as pd
import re
from dotenv import load_dotenv
import os
from google import genai
import json
import time
from tqdm import tqdm

from models import FormatoResposta

load_dotenv()

True

In [3]:
# Captura pedido de música
query = input("O que você gostaria de ouvir hoje? ")

## Playlist gerada a partir de uma única requisição ao Gemini (LLM)

In [ ]:
def monta_playlist(texto):
    """
    Função que realiza uma requisição de montagem de playlist diretamente para o Gemini.
    """
    
    prompt = f"""
    Você é um especialista em música e construção de playlists. 
    Você deve analisar o seguinte pedido de playlist e construir uma seleção de músicas que se encaixem no tema solicitado.
    Refira-se EXCLUSIVAMENTE ao pedido fornecido para criar a playlist.
    A playlist deve ser composta por 10 músicas, com o nome da música, do artista e possível featuring.
    A resposta deve ser uma lista em formato JSON, com os seguintes campos:
    - nome_musica: Nome da música
    - artista: Nome do artista
    - featuring: Nome do artista convidado (caso exista)

    Pedido do usuário: {texto}
    """

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': list[FormatoResposta],
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    return resposta

# Resposta do Gemini
resposta = monta_playlist(query)
# Converte a resposta em um dicionário
resposta_dict = json.loads(resposta.text)
# Cria um DataFrame a partir da lista de dicionários
df = pd.DataFrame(resposta_dict)
# Renomeia as colunas
df.rename(columns={'nome_musica': 'Nome da Música', 'artista': 'Artista', 'featuring': 'Featuring'}, inplace=True)
df

,Nome da Música,Artista,Featuring
0,Umbrella,Rihanna,Jay-Z
1,Crazy in Love,Beyoncé,Jay-Z
2,Family Affair,Mary J. Blige,None
3,Yeah!,Usher,Lil Jon & Ludacris
4,Hot in Herre,Nelly,None
5,Gold Digger,Kanye West,Jamie Foxx
6,Empire State of Mind,Jay-Z,Alicia Keys
7,In Da Club,50 Cent,None
8,Lose Control,Missy Elliott,Ciara & Fatman Scoop
9,My Boo,Usher,Alicia Keys


## Playlist gerada a partir de interações entre agentes

In [4]:
from agents import PlaylistGenerator, Validator

agente = PlaylistGenerator()

TypeError: PlaylistGenerator.__init__() missing 2 required positional arguments: 'config' and 'model'

## Playlist gerada a partir de BERT embeddings

In [4]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-large-cased')

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [10]:
import pandas as pd
df = pd.read_csv('data/songs_filtrado.csv')
X = df["lyrics"]
y = df['tag']

from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

# Utiliza a GPU para fazer as operações, fazendo muito mais rápido para modelos grandes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_embeddings(text, model, tokenizer):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[0, 0, :].cpu()

        # Para nao gastar tanta memoria
        del inputs, outputs
    return cls_embedding

batch_size = 500
for batch_i, inicio in enumerate(tqdm(range(0, len(X), batch_size))):
    # Se chegou ao fim, garante nao passar o index
    fim = inicio+batch_size if inicio+batch_size < len(X) else len(X)

    # Pega pequena amostra, para evitar estourar a memoria RAM (kk)
    batch = X.iloc[inicio:fim]

    embeddings = []
    for i in range(len(batch)):
        e = get_embeddings(batch.iloc[i], model, tokenizer)
        # print(f"{batch.iloc[i]} :{e.detach().numpy()}")
        embeddings.append(e.detach().numpy())
    
    embeddings = np.array(embeddings)

    # Salva os embeddings por batch, depois vamos juntar tudo
    np.save(f'embeddings/bert_emb_{batch_i}.npy', embeddings) 

    # Libera variaveis pesadas
    del embeddings
    # Por usar GPU
    torch.cuda.empty_cache()

100%|██████████| 575/575 [51:22<00:00,  5.36s/it]


In [ ]:
embeddings = np.array([])

for file in tqdm(os.listdir("./embeddings")):
    f = f"./embeddings/{file}"

    embeddings_load = np.load(f)
    embeddings = np.append(embeddings, embeddings_load)

np.save(f'bert_final.npy', embeddings)

del embeddings_load
del embeddings

100%|██████████| 575/575 [02:52<00:00,  3.33it/s]
